In [1]:
!nvidia-smi

Sun Jan  5 20:08:22 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1080 Ti     On  |   00000000:03:00.0 Off |                  N/A |
| 21%   39C    P8             10W /  250W |       1MiB /  11264MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch
torch.cuda.empty_cache()  # Free up unutilized memory

In [3]:
from src.segmentation.framework_handlers import ultralytics_handler
ultralytics_handler.validate_version_and_gpu()

Ultralytics 8.3.56 🚀 Python-3.9.19 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11165MiB)
Setup complete ✅ (12 CPUs, 125.2 GB RAM, 13.9/336.4 GB disk)


### Prepare Dataset
In this section, the dataset along with YOLO annotations is processed to conform to the input structure required by YOLO. The YOLO format necessitates organizing the dataset into two main directories, `images` and `labels`. Each of these directories is further divided into two subdirectories, `train` and `val`, for training and validation datasets respectively. This structure facilitates YOLO's training and validation processes.
WE can process the data by two options:
- Single SegmentsAI dataset
- Two SegmentsAI datasets, one for train and the other for test

Option 1: prepare data using a single dataset by splitting to train and test

### Ultralytics Settings

In here we can modify the settings of the yolo runs. 

In [4]:
from ultralytics import settings
import config

# Update a setting
yolo_runs_dir = "/home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/runs"
settings.update(
    {
        'runs_dir': yolo_runs_dir,
        'weights_dir': f"{yolo_runs_dir}/weights",
        'datasets_dir': f"{yolo_runs_dir}/datasets",
    }
)

# View all settings
print(settings)

JSONDict("/home/etaylor/.config/Ultralytics/settings.json"):
{
  "settings_version": "0.0.6",
  "datasets_dir": "/home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/runs/datasets",
  "weights_dir": "/home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/runs/weights",
  "runs_dir": "/home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/runs",
  "uuid": "01de958df926b4e70f7bb00e8bdc993b48b07bc8d023fc13b26874ee84610e6f",
  "sync": true,
  "api_key": "",
  "openai_api_key": "",
  "clearml": true,
  "comet": true,
  "dvc": true,
  "hub": true,
  "mlflow": true,
  "neptune": true,
  "raytune": true,
  "tensorboard": true,
  "wandb": false,
  "vscode_msg": true
}


In [5]:
import torch
import gc

gc.collect()
torch.cuda.empty_cache()

In [7]:
import os
import json
from ultralytics import YOLO
import torch
import gc

# Path to the dataset YAML configuration
yaml_file_path = "/home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/yaml/stigmas_dataset_yolo.yaml"

# Paths to model checkpoints and fine-tuned checkpoint storage
pre_trained_checkpoint_dir = "/home/etaylor/code_projects/thesis/checkpoints/stigmas_segmentation/yolo/pre_trained"
fine_tuned_checkpoint_dir = "/home/etaylor/code_projects/thesis/checkpoints/stigmas_segmentation/yolo/fine_tuned"

# YOLO segmentation model checkpoints
available_models_checkpoints = {
    # "yolov8n-seg": os.path.join(pre_trained_checkpoint_dir, "yolov8n-seg.pt"),
    # "yolov8s-seg": os.path.join(pre_trained_checkpoint_dir, "yolov8s-seg.pt"),
    # "yolov8m-seg": os.path.join(pre_trained_checkpoint_dir, "yolov8m-seg.pt"),
    # "yolov8l-seg": os.path.join(pre_trained_checkpoint_dir, "yolov8l-seg.pt"),
    # "yolov8x-seg": os.path.join(pre_trained_checkpoint_dir, "yolov8x-seg.pt"),
    "yolov11n-seg": os.path.join(pre_trained_checkpoint_dir, "yolo11n-seg.pt"),
    "yolov11s-seg": os.path.join(pre_trained_checkpoint_dir, "yolo11s-seg.pt"),
    "yolov11m-seg": os.path.join(pre_trained_checkpoint_dir, "yolo11m-seg.pt"),
    # "yolov11l-seg": os.path.join(pre_trained_checkpoint_dir, "yolo11l-seg.pt"),
    # "yolov11x-seg": os.path.join(pre_trained_checkpoint_dir, "yolo11x-seg.pt")
}

# Ensure fine-tuned checkpoint directory exists
os.makedirs(fine_tuned_checkpoint_dir, exist_ok=True)

# Train and fine-tune each model
results_summary = {}

for model_name, checkpoint_name in available_models_checkpoints.items():
    print(f"Training model: {model_name}")

    # Load the pre-trained model checkpoint from the original path
    model = YOLO(checkpoint_name)

    # Train the model
    results = model.train(data=yaml_file_path, epochs=50, imgsz=640, batch=2)

    # Validate the trained model
    valid_results = model.val()
    print(f"Validation results for {model_name}:", valid_results)

    # Save the fine-tuned checkpoint
    fine_tuned_checkpoint_path = os.path.join(fine_tuned_checkpoint_dir, f"{model_name}_fine_tuned.pt")
    model.save(fine_tuned_checkpoint_path)
    print(f"Fine-tuned model saved to: {fine_tuned_checkpoint_path}")

    # Save the results
    results_summary[model_name] = valid_results
    
    gc.collect()
    torch.cuda.empty_cache()

# Save all results to a JSON file
summary_file_path = os.path.join(fine_tuned_checkpoint_dir, "fine_tuning_results.json")

with open(summary_file_path, "w") as f:
    json.dump(results_summary, f)

print("Fine-tuning complete. Results saved.")


Training model: yolov11n-seg
Ultralytics 8.3.56 🚀 Python-3.9.19 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11165MiB)
engine/trainer: task=segment, mode=train, model=/home/etaylor/code_projects/thesis/checkpoints/stigmas_segmentation/yolo/pre_trained/yolo11n-seg.pt, data=/home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/yaml/stigmas_dataset_yolo.yaml, epochs=50, time=None, patience=100, batch=2, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train28, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visuali

train: Scanning /sise/home/etaylor/code_projects/thesis/segments/etaylor_stigmas_dataset/yolo_formatted/labels/train.cache... 87 images, 0 backgrounds, 0 corrupt: 100%|██████████| 87/87 [00:00<?, ?it/s]
/home/etaylor/.conda/envs/detectron_fastai/lib/python3.9/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
val: Scanning /sise/home/etaylor/code_projects/thesis/segments/etaylor_stigmas_dataset/yolo_formatted/labels/val.cache... 25 images, 0 backgrounds, 0 corrupt: 100%|██████████| 25/25 [00:00<?, ?it/s]
/home/etaylor/.conda/envs/detectron_fastai/lib/python3.9/site-packages/torch/utils/data/data

Plotting labels to /home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/runs/segment/train28/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 90 weight(decay=0.0), 101 weight(decay=0.0005), 100 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/runs/segment/train28
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


       1/50      2.08G      2.019      4.137      3.121      1.816         17        640: 100%|██████████| 44/44 [00:06<00:00,  6.82it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:00<00:00, 10.95it/s]

                   all         25        190     0.0212      0.837      0.154     0.0573    0.00733      0.289     0.0157    0.00332


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  5.85it/s]


                   all         25        190      0.338      0.421      0.316      0.143      0.245      0.247      0.135     0.0286
Speed: 0.2ms preprocess, 21.6ms inference, 0.1ms loss, 7.7ms postprocess per image
Results saved to /home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/runs/segment/train30
Ultralytics 8.3.56 🚀 Python-3.9.19 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11165MiB)
YOLO11m-seg summary (fused): 330 layers, 22,336,083 parameters, 0 gradients, 123.0 GFLOPs


val: Scanning /sise/home/etaylor/code_projects/thesis/segments/etaylor_stigmas_dataset/yolo_formatted/labels/val.cache... 25 images, 0 backgrounds, 0 corrupt: 100%|██████████| 25/25 [00:00<?, ?it/s]
/home/etaylor/.conda/envs/detectron_fastai/lib/python3.9/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:03<00:00,  4.11it/s]


                   all         25        190      0.358      0.411      0.323      0.143      0.245      0.247      0.134     0.0281
Speed: 1.1ms preprocess, 32.7ms inference, 0.0ms loss, 8.9ms postprocess per image
Results saved to /home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/runs/segment/train302
Validation results for yolov11m-seg: ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fc416499bb0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0

TypeError: Object of type SegmentMetrics is not JSON serializable

### Perform Infrence with the model

In [8]:
import cv2

image = cv2.imread("/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train/v0.1/IMG_2198_p12.png")

# Perform inference
results = model.predict(image)


0: 640x640 (no detections), 19.5ms
Speed: 12.6ms preprocess, 19.5ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
